In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [16]:
import time
from tqdm import tqdm

<IPython.core.display.Javascript object>

In [1]:
from steamship import Steamship, PluginInstance, File

from steamship import Block, Tag
from steamship.base.tasks import TaskState

In [2]:
client = Steamship(profile="staging")


def get_steamship_client():
    return client

In [5]:
FILE_IMPORTER_HANDLE = "youtube-file-importer"

file_importer = PluginInstance.create(
    client,
    plugin_handle=FILE_IMPORTER_HANDLE,
    config={},
).data
print("importer", file_importer.id)

importer E2760A66-B520-46EB-9C28-224B7815979C


<IPython.core.display.Javascript object>

In [6]:
BLOCKIFIER_HANDLE = "assemblyai-s2t-blockifier"

s2t_blockifier = PluginInstance.create(
    client,
    plugin_handle=BLOCKIFIER_HANDLE,
    config={},
).data
print("blockifier", s2t_blockifier.id)

blockifier A48D54AA-A613-491A-B724-BC055B528563


<IPython.core.display.Javascript object>

In [ ]:
[
    ("https://youtu.be/G0Cp7DrvNLQ", "Randall Carlson"),
    ("https://youtu.be/fTAPaHcLAhw", "Wiz Khalifa"),
    ("https://youtu.be/Oyb1xz7waY8", "Macaulay Culkin"),
    ("https://youtu.be/_Z-RdKb1TPg", "Theo Von"),
    ("https://youtu.be/pwaWilO_Pig", "Matthew Walker"),
    ("https://youtu.be/A_7RmheFRz0", "Jamie Foxx"),

]

In [31]:
podcasts = [
    ("https://youtu.be/7MNv4_rTkfU", "Mike Tyson"),
    ("https://youtu.be/vIeFt88Hm8s", "Jordan Peterson"),
    ("https://youtu.be/9Xc7DN-noAc", "Jordan Peterson"),
    ("https://youtu.be/qxOeWuAHOiw", "Kanye West"),
    ("https://youtu.be/2O-iLk1G_ng", "Bernie Sanders"),
]

<IPython.core.display.Javascript object>

In [32]:
file_futures = []
for podcast_url, podcast_guest in tqdm(podcasts):
    file = File.create(
        client, plugin_instance=file_importer.handle, url=podcast_url
    ).data
    file_futures.append(
        (
            file.blockify(plugin_instance=s2t_blockifier.handle),
            file,
            podcast_url,
            podcast_guest,
        )
    )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:10<00:00, 14.09s/it]


<IPython.core.display.Javascript object>

In [39]:
skipped_urls = []
while len(skipped_urls) < len(file_futures):
    print(len(skipped_urls), len(file_futures))
    print(".", end="")
    for file_future, file, podcast_url, podcast_guest in file_futures:
        if podcast_url not in skipped_urls:
            file_future.wait(max_timeout_s=2, retry_delay_s=1)
            if file_future.task.state in (TaskState.succeeded, TaskState.failed):
                print(file.id, file_future.task.state)
                if file_future.task.state == TaskState.succeeded:
                    try:
                        skipped_urls.append(podcast_url)
                        file = file.refresh().data
                        f = File.create(
                            client,
                            tags=[
                                Tag.CreateRequest(kind="youtube_url", name=podcast_url),
                                Tag.CreateRequest(kind="guest", name=podcast_guest),
                            ],
                            blocks=file.blocks,
                        ).data
                        tags = f.blocks[0].tags
                        print(len(tags), {tag.kind for tag in tags})
                        file.delete()
                    except Exception:
                        pass

    time.sleep(5)

0 5
.F60582D1-9CFB-4CE3-BEF2-812F772A8DDF succeeded


ERROR:root:Client received error from server: [ObjectNotFound]
Unable to find object id=F60582D1-9CFB-4CE3-BEF2-812F772A8DDF handle=?


F46788A9-0CD2-4176-9639-BC4E460644F2 succeeded


ERROR:root:Client received error from server: [ObjectNotFound]
Unable to find object id=F46788A9-0CD2-4176-9639-BC4E460644F2 handle=?


27E2748B-0DED-4440-80AD-0C38756FB8B4 succeeded


...

EBEE09DA-4707-4CF8-AA51-C7EBB203C0A2 succeeded


ERROR:root:Client received error from server: [ObjectNotFound]
Unable to find object id=EBEE09DA-4707-4CF8-AA51-C7EBB203C0A2 handle=?


9316142C-A712-4EF9-8250-D2E2A894B8A2 succeeded


ERROR:root:Client received error from server: [ObjectNotFound]
Unable to find object id=9316142C-A712-4EF9-8250-D2E2A894B8A2 handle=?


<IPython.core.display.Javascript object>